In [ ]:
import tkinter as tk
from tkinter import simpledialog
from playsound import playsound
import threading
import time
import random

vocabulary = {
    'Hello: Konnichiwa こんにちは': 'C:/Users/Gauri/Downloads/V1.mp3',
    'Yes: Hai はい': 'C:/Users/Gauri/Downloads/V2.mp3',
    'No: ie の': 'C:/Users/Gauri/Downloads/V3.mp3',
    'One: Ichi いち': 'C:/Users/Gauri/Downloads/V4.mp3',
    'Two: Ni に': 'C:/Users/Gauri/Downloads/V5.mp3',
    'Three: San さん': 'C:/Users/Gauri/Downloads/V6.mp3',
    'Four: Yon よん': 'C:/Users/Gauri/Downloads/V7.mp3',
    'Five: Go ご': 'C:/Users/Gauri/Downloads/V8.mp3',
    'Book: Hon ほん': 'C:/Users/Gauri/Downloads/V9.mp3',
    'Car: Kuruma くるま': 'C:/Users/Gauri/Downloads/V10.mp3',
    'Phone: Keitai けいたい': 'C:/Users/Gauri/Downloads/V11.mp3',
    'Water: Mizu みず': 'C:/Users/Gauri/Downloads/V12.mp3',
    'Food: Tabemono たべもの': 'C:/Users/Gauri/Downloads/V13.mp3',
    'Eat: Taberu たべる': 'C:/Users/Gauri/Downloads/V14.mp3',
    'Drink: Nomu のむ': 'C:/Users/Gauri/Downloads/V15.mp3',
    'Go: Iku いく': 'C:/Users/Gauri/Downloads/V16.mp3',
    'See/Look: Miru みる': 'C:/Users/Gauri/Downloads/V17.mp3',
    'Speak: Hanasu はなす': 'C:/Users/Gauri/Downloads/V18.mp3'
}

phrases = {
    'Good Morning: Ohayo Gozaimasu - おはようございます': 'C:/Users/Gauri/Downloads/P1.mp3',
    'Good Afternoon: Konnichiwa - こんにちは': 'C:/Users/Gauri/Downloads/P2.mp3',
    'Good Evening: Konbanwa - こんばんは': 'C:/Users/Gauri/Downloads/P3.mp3',
    'Good Night: Oyasuminasai - おやすみなさい': 'C:/Users/Gauri/Downloads/P4.mp3',
    'Sorry/Excuse Me: Sumimasen - すみません': 'C:/Users/Gauri/Downloads/P5.mp3',
    'Goodbye: Sayonara - さようなら': 'C:/Users/Gauri/Downloads/P6.mp3',
    'Please: Onegaishimasu - おねがいします': 'C:/Users/Gauri/Downloads/P7.mp3',
    'Thank You: Arigatou - ありがとう': 'C:/Users/Gauri/Downloads/P8.mp3',
    'You are Welcome: Dou itashimashite - どういたしまして': 'C:/Users/Gauri/Downloads/P9.mp3',
    'How are you: Ogenki desu ka? - おげんきですか？': 'C:/Users/Gauri/Downloads/P10.mp3',
    'Yes I am fine: Hai, genki desu - はい、げんきです': 'C:/Users/Gauri/Downloads/P11.mp3',
    'Take Care: Ki o tsukete - き　を　つけて': 'C:/Users/Gauri/Downloads/P12.mp3',
    'Nice to meet you: Hajimemashite - はじめまして': 'C:/Users/Gauri/Downloads/P13.mp3',
    'My name is: Watashi no namae wa (name) desu - わたしのなまえは（name）です': 'C:/Users/Gauri/Downloads/P14.mp3',
    'I am From: Kara kimashita - （くに）からきました': 'C:/Users/Gauri/Downloads/P15.mp3'
}

is_paused = False
stop_learning = False
current_audio = None

def get_random_color():
    return random.choice(['#FF6347', '#4682B4', '#32CD32', '#FFD700', '#6A5ACD', '#FF69B4'])

def play_audio(path):
    threading.Thread(target=playsound, args=(path,), daemon=True).start()

def play_current_audio():
    if current_audio:
        play_audio(current_audio)

def pause_learning():
    global is_paused
    is_paused = True

def resume_learning():
    global is_paused
    is_paused = False

def stop_learning_func():
    global stop_learning
    stop_learning = True

def ask_integer(prompt):
    result = None
    def ask():
        nonlocal result
        result = simpledialog.askinteger("Input", prompt)
    root.after(0, ask)
    while result is None:
        root.update()
    return result

def learn_items(data_dict):
    def run_learning():
        global is_paused, stop_learning, current_audio
        stop_learning = False
        count = ask_integer("Enter number to learn (0 for all):")
        if count is None:
            return
        if count == 0 or count > len(data_dict):
            count = len(data_dict)

        show_audio_controls()
        items = list(data_dict.items())[:count]

        for word, audio in items:
            while is_paused:
                root.update()
            if stop_learning:
                text_display.config(text="Stopped", fg="red", bg="#fff")
                hide_audio_controls()
                return
            current_audio = audio
            text_display.config(text=word, fg=get_random_color(), bg="#fff") 
            root.update()
            time.sleep(4)

        text_display.config(text="Done!", fg="green", bg="#fff")
        hide_audio_controls()

    threading.Thread(target=run_learning, daemon=True).start()

def quiz(data_dict):
    def run_quiz():
        score = 0
        keys = list(data_dict.keys())
        random.shuffle(keys)
        
        count = ask_integer("Enter number of quiz questions (max 15):")
        if count is None:
            return
        if count > len(data_dict):
            count = len(data_dict)
        if count > 15:  
            count = 15

        for i, word in enumerate(keys[:count]):
            question = f"What is the Japanese for: {word.split(':')[0]}?"
            text_display.config(text=question, fg="black", bg="#fff")  
            root.update()
            
            answer = simpledialog.askstring("Quiz", question)
            correct_answer = word.split(':')[1].strip()
            
            if answer and answer.lower() in correct_answer.lower():
                score += 1

        text_display.config(text=f"Quiz Completed! Your score: {score}/{count}", fg="green", bg="#fff")

    run_quiz()

# GUI Setup
root = tk.Tk()
root.title("Language Learning Flashcards")
root.geometry("500x550")
root.configure(bg="#f0f0f0")

tk.Label(root, text="Japanese Flashcards", font=("Helvetica", 18, "bold"), bg="#f0f0f0").pack(pady=10)

# Flashcard Style Frame
content_frame = tk.Frame(root, bg="#f0f0f0")
content_frame.pack(pady=10)

text_display = tk.Label(
    content_frame,
    text="",
    font=("Helvetica", 16, "bold"),
    width=40,
    height=6,
    bg="white",
    fg="black",
    wraplength=380,
    justify="center",
    relief="solid",
    bd=2
)
text_display.pack(pady=(0, 10), padx=10)

control_frame = tk.Frame(content_frame, bg="#f0f0f0")
control_frame.pack()

play_btn = tk.Button(control_frame, text="▶ Play Audio", command=play_current_audio)
pause_btn = tk.Button(control_frame, text="⏸ Pause", command=pause_learning)
resume_btn = tk.Button(control_frame, text="⏵ Resume", command=resume_learning)
stop_btn = tk.Button(control_frame, text="⏹ Stop", command=stop_learning_func)

def show_audio_controls():
    play_btn.pack(side="left", padx=5)
    pause_btn.pack(side="left", padx=5)
    resume_btn.pack(side="left", padx=5)
    stop_btn.pack(side="left", padx=5)

def hide_audio_controls():
    play_btn.pack_forget()
    pause_btn.pack_forget()
    resume_btn.pack_forget()
    stop_btn.pack_forget()

# Buttons
tk.Button(root, text="📘 Learn Vocabulary", command=lambda: learn_items(vocabulary)).pack(pady=5)
tk.Button(root, text="📗 Learn Phrases", command=lambda: learn_items(phrases)).pack(pady=5)
tk.Button(root, text="📝 Quiz Vocabulary", command=lambda: quiz(vocabulary)).pack(pady=5)
tk.Button(root, text="📝 Quiz Phrases", command=lambda: quiz(phrases)).pack(pady=5)

root.mainloop()